### Removing first n volumes of BOLD time series: 
This is a product of the machine’s magnetic field first lining up the protons in the water molecules in the brain, before they have gone into a stead stat. 
<br>
To remove the thermal noise effect we remove first 5 volumes.
<br>
In addition, to avoide the edge effect of temporal filtering, we discarded the last 5 volumes.
<br>
Second code will remove the motion parameters of discarded volumes form bold_mc.par file.

-----------------------------------------------------------
Script written by Sepideh Tabrik & Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
from pathlib import Path
import os

dummy_scan = 5  # Number of volumes to remove (edit here)

data_path = Path('/mnt/d/Data/Human/ObjectCategorization/analysis2')
bold_dirs = list(data_path.glob('sub-*/ses-0*/func/rest*/*bandpassed.nii.gz'))

def run_command(command):
    """Execute a shell command with error handling."""
    exit_code = os.system(command)
    if exit_code != 0:
        print(f"Error executing: {command}")

for curr_bold in bold_dirs:
    curr_sub = curr_bold.parent
    print(f'Discarding the first {dummy_scan} and the last {dummy_scan} volumes of dataset: {curr_sub}\n')

    filename = curr_bold.stem  # Extract filename without extension

    os.chdir(curr_sub)

    # Get number of volumes
    nVolume = os.popen(f"fslnvols {filename}").read().strip()
    if nVolume.isdigit():
        nVolume = int(nVolume) - 2 * dummy_scan
        run_command(f"fslroi {filename} preprocessed_func_data {dummy_scan} {nVolume}")
    
        # Alternative processing with 3dTcat (commented out)
        # run_command(f"3dTcat -prefix preprocessed_data_bandpassed_chopped.nii.gz preprocessed_data_bandpassed.nii.gz[{dummy_scan}..{nVolume}]")
        # run_command(f"3dTcat -prefix preprocessed_data_highpassed_chopped.nii.gz preprocessed_data_highpassed.nii.gz[{dummy_scan}..{nVolume}]")
    else:
        print(f"Warning: Could not determine volume count for {filename}")


## Removing n voluems for estimated-motion parameters

In [ ]:
from pathlib import Path
import os

dummy_scan = 5  # Number of volumes to remove (edit here)

data_path = Path('/mnt/d/Data/Human/ObjectCategorization/analysis2')
sub_dirs = list(data_path.glob('sub*/ses-*/func/rest*/mc'))

for curr_sub in sub_dirs:  # Skip the first item
    print(f'Discarding the first {dummy_scan} and the last {dummy_scan} volumes of mc: {curr_sub}\n')

    # Change to the current directory
    os.chdir(curr_sub)  # Use os.chdir() instead of Path.chdir()

    mc_file = curr_sub / "bold_mcf.par"
    mc_out = curr_sub / "bold_mcf_chopped.par"

    # Count number of lines
    num_lines = sum(1 for _ in mc_file.open())

    # Process and write valid lines
    with mc_file.open() as infile, mc_out.open("w") as outfile:
        for pos, line in enumerate(infile):
            if dummy_scan <= pos < num_lines - dummy_scan:
                outfile.write(line)
